<a href="https://colab.research.google.com/github/mathubhalan/BERT-NLP/blob/master/BERT_Sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

You should consider upgrading via the 'C:\Users\sjvd2\Anaconda3\python.exe -m pip install --upgrade pip' command.



  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893262 sha256=d9ba4408ad801f364c4d529883ab8e99cc9f795053a76ecdae789bc30c92b78b
  Stored in directory: c:\users\sjvd2\appdata\local\pip\cache\wheels\69\09\d1\bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


# Exploratory Data Analysis and Preprocessing


In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
df= pd.read_csv("Train.csv")
df.set_index('Text_ID', inplace=True)

In [4]:
df.head()

,Product_Description,Product_Type,Sentiment
Text_ID,,,
3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
6254,RT @mention Line for iPad 2 is longer today th...,9,2
8212,Crazy that Apple is opening a temporary store ...,9,2
4422,The lesson from Google One Pass: In this digit...,9,2
5526,RT @mention At the panel: &quot;Your mom has a...,9,2


In [5]:
df.Sentiment.value_counts()

2    3765
3    2089
1     399
0     111
Name: Sentiment, dtype: int64

Class - Represents various sentiments
0. Cannot Say
1. Negative
2. Positive
3. No Sentiment

# Training/Validation Split

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.Sentiment.values, 
                                                  test_size=0.25, 
                                                  random_state=99, 
                                                  stratify=df.Sentiment.values)

In [9]:
y_train

array([2, 2, 2, ..., 1, 2, 2], dtype=int64)

Check to validate if we have equal data in all classes


In [10]:
df['data_type'] = ['not_set']*df.shape[0]

In [11]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [12]:
df.head()

,Product_Description,Product_Type,Sentiment,data_type
Text_ID,,,,
3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2,val
6254,RT @mention Line for iPad 2 is longer today th...,9,2,train
8212,Crazy that Apple is opening a temporary store ...,9,2,train
4422,The lesson from Google One Pass: In this digit...,9,2,train
5526,RT @mention At the panel: &quot;Your mom has a...,9,2,train


In [13]:
df.columns

Index(['Product_Description', 'Product_Type', 'Sentiment', 'data_type'], dtype='object')

In [14]:
df.groupby(['Sentiment', 'data_type']).count()

Product_Description  Product_Type
Sentiment data_type                                   
0         train                       83            83
          val                         28            28
1         train                      299           299
          val                        100           100
2         train                     2824          2824
          val                        941           941
3         train                     1567          1567
          val                        522           522

In [15]:
df[df.data_type=='train'][['Product_Description','Product_Type']].values

array([['RT @mention Line for iPad 2 is longer today than yesterday. #SXSW  // are you getting in line again today just for fun?',
        9],
       ['Crazy that Apple is opening a temporary store in Austin tomorrow to handle the rabid #sxsw eye pad too seekers.',
        9],
       ['The lesson from Google One Pass: In this digital environment, users want to purchase across every platform with one tool. #sxsw #elonsxsw',
        9],
       ...,
       ['SXSW iPhone app is awesome, but iPad app crashes every time. #sxsw',
        7],
       ['RT @mention RT @mention Download 20+ free tracks from @mention Music Sampler @mention including @glove! {link} #SXSW',
        9],
       ['#Google launching a &quot;major&quot; new social network at #sxsw ... Wonder what that can be...',
        9]], dtype=object)

# Loading Tokenizer and Encoding our Data

In [16]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [18]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Product_Description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\sjvd2\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Product_Description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [20]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Sentiment.values)

In [21]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [22]:
print (f"shape of train is :{len(dataset_train)}")
print (f"shape of val is :{len(dataset_val)}")

shape of train is :4773
shape of val is :1591


# Setting up BERT Pretrained Model

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders

In [25]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [26]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Setting Up Optimiser and Scheduler

In [27]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [28]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [29]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Defining our Performance Metrics

In [30]:
import numpy as np
from sklearn.metrics import f1_score

In [31]:
label_dict ={'Cannot Say':0,'Negative':1,'Postive':2,'No Sentiment':3}
label_dict

{'Cannot Say': 0, 'Negative': 1, 'Postive': 2, 'No Sentiment': 3}

In [32]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [33]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating our Training Loop

In [34]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [36]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [55]:
_, predictions, true_vals = evaluate(dataloader_validation)


In [57]:
predictions

array([[-1.61608   , -1.8265791 ,  3.5225694 , -0.646289  ],
       [-1.7421474 , -2.3777046 ,  3.785414  , -0.6429743 ],
       [-1.5066649 , -1.8749869 , -0.0077248 ,  3.7427683 ],
       ...,
       [-1.7219838 , -1.8906404 ,  3.097601  , -0.4756199 ],
       [-2.2620668 , -2.6661584 ,  2.0910578 ,  1.7683979 ],
       [-1.6994187 , -1.9174868 ,  3.4385614 , -0.95996207]],
      dtype=float32)

In [56]:
accuracy_per_class(predictions, true_vals)

Class: Cannot Say
Accuracy: 0/17

Class: Negative
Accuracy: 21/60

Class: Postive
Accuracy: 446/565

Class: No Sentiment
Accuracy: 214/313

